###User_Based_Collaborative_Filtering.ipynb

##This notebook demonstrates how to build an User-Based Collaborative Filtering (IBCF) Recommendation System using the Surprise library.



In [63]:
!pip install "numpy<2"  # required for surprise to work as it doesnt work with numpy 2


##### Mounting Google Drive to access files


In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
import pandas as pd

df=pd.read_csv('/content/drive/MyDrive/0.Latest_DS_Course/RS/data/ratings_sub.csv', encoding = "ISO-8859-1")
print(df)

        userId  movieId  rating   timestamp  \
0         3218     3889     1.0  1172532894   
1         3663     3889     1.0  1044474348   
2         3704     3889     3.0   971391538   
3         8877     3889     1.0  1050744366   
4         9599     3889     0.5  1378056755   
...        ...      ...     ...         ...   
487464  130784   109159     3.5  1427063644   
487465  134800    97732     4.0  1351051540   
487466  134800    87644     4.5  1308552954   
487467  134800    99171     4.0  1356252918   
487468  134800   101581     4.0  1364789324   

                                                    title  \
0              Highlander: Endgame (Highlander IV) (2000)   
1              Highlander: Endgame (Highlander IV) (2000)   
2              Highlander: Endgame (Highlander IV) (2000)   
3              Highlander: Endgame (Highlander IV) (2000)   
4              Highlander: Endgame (Highlander IV) (2000)   
...                                                   ...   
487464  

In [66]:
# Assign DataFrame to ratings variable
ratings = df

In [67]:
ratings.head()

,userId,movieId,rating,timestamp,title,genres,year
0,3218,3889,1.0,1172532894,Highlander: Endgame (Highlander IV) (2000),Action|Adventure|Fantasy,2000.0
1,3663,3889,1.0,1044474348,Highlander: Endgame (Highlander IV) (2000),Action|Adventure|Fantasy,2000.0
2,3704,3889,3.0,971391538,Highlander: Endgame (Highlander IV) (2000),Action|Adventure|Fantasy,2000.0
3,8877,3889,1.0,1050744366,Highlander: Endgame (Highlander IV) (2000),Action|Adventure|Fantasy,2000.0
4,9599,3889,0.5,1378056755,Highlander: Endgame (Highlander IV) (2000),Action|Adventure|Fantasy,2000.0


In [68]:
ratings.shape

(487469, 7)

Scikit-surprise (often referred to simply as Surprise) is a Python scikit (like scikit-learn) built specifically for building and analyzing recommender systems. It is designed to handle collaborative filtering tasks efficiently, making it an excellent tool for experimenting with recommendation algorithms.

##Surprise supports:

* User-based and item-based collaborative filtering

* Matrix factorization methods like SVD, SVD++, NMF

* Model selection via cross-validation

* Prediction accuracy metrics like RMSE, MAE

In [69]:
!pip install scikit-surprise

In [70]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487469 entries, 0 to 487468
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     487469 non-null  int64  
 1   movieId    487469 non-null  int64  
 2   rating     487469 non-null  float64
 3   timestamp  487469 non-null  int64  
 4   title      487469 non-null  object 
 5   genres     487469 non-null  object 
 6   year       487469 non-null  float64
dtypes: float64(2), int64(3), object(2)
memory usage: 26.0+ MB


### # Convert userId and movieId to string (required for Surprise library)


In [71]:
ratings.userId = ratings.userId.astype(str)

In [72]:
ratings.movieId = ratings.movieId.astype(str)

In [73]:
# Check number of unique users and movies
print("Unique users:", ratings['userId'].nunique())
print("Unique movies:", ratings['movieId'].nunique())

Unique users: 2827
Unique movies: 6656


In [74]:
# View most active users (those with most ratings)
print(ratings['userId'].value_counts().head())

userId
3218     200
75694    200
61382    200
47594    200
29990    200
Name: count, dtype: int64


In [75]:
# View most rated movies
print(ratings['movieId'].value_counts().head())

movieId
4993    2481
4306    2356
5952    2338
7153    2235
3578    2226
Name: count, dtype: int64


In [76]:
# Install surprise library if not already installed
# !pip install surprise

In [77]:
import numpy as np
print(np.__version__)


1.26.4


#### Import Surprise modules


In [78]:
from surprise import Dataset,Reader

###Define the format of the input data for Surprise


In [79]:
reader = Reader(rating_scale=(1,5))  # Ratings range from 1 to 5

#### Load data into Surprise dataset format


In [80]:
data = Dataset.load_from_df(ratings[['userId', 'title', 'rating']], reader)

| Surprise Concept | Your Columns |
| ---------------- | ------------ |
| `uid` (user ID)  | `userId`     |
| `iid` (item ID)  | `title`      |
| `rating`         | `rating`     |


* user_id (from your original DataFrame)

* uid (internal user ID used by Surprise)

* item_title (like movie title)

* iid (internal item ID used by Surprise)



In [81]:
data

#### Split dataset into train and test sets (75%-25% split)


In [82]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25, random_state = 123)

In [83]:
# To build on full data

#trainset = data.build_full_trainset()

In [84]:
trainset

In [85]:
# raw id - original userid and title

# inner id - mapped values created by surprise library

In [86]:
# View internal training data structure
print("Trainset size (interactions):", trainset.n_ratings)

Trainset size (interactions): 365601


In [87]:
# Understanding raw vs internal IDs used by Surprise
user_records = trainset.ur   # All user interactions

In [88]:
user_records[0]

[(0, 3.0),
 (195, 4.0),
 (1066, 3.5),
 (999, 3.5),
 (237, 3.0),
 (1577, 3.0),
 (932, 2.0),
 (247, 4.5),
 (2215, 3.0),
 (221, 4.0),
 (745, 3.0),
 (133, 3.0),
 (249, 3.0),
 (1065, 2.5),
 (255, 3.5),
 (167, 4.0),
 (586, 3.5),
 (1234, 4.0),
 (259, 4.5),
 (729, 2.5),
 (236, 3.5),
 (181, 3.5),
 (3245, 3.5),
 (1014, 3.0),
 (577, 5.0),
 (2789, 3.5),
 (91, 4.0),
 (10, 4.0),
 (19, 3.5),
 (274, 4.0),
 (2135, 3.0),
 (1419, 3.5),
 (695, 4.0),
 (1373, 3.5),
 (850, 3.0),
 (334, 4.0),
 (2759, 3.0),
 (222, 3.0),
 (37, 4.0),
 (380, 2.5),
 (544, 4.0),
 (542, 4.5),
 (1135, 5.0),
 (650, 5.0),
 (4625, 3.5),
 (341, 1.0),
 (780, 4.0),
 (2371, 3.0),
 (661, 4.0),
 (4742, 4.5),
 (1660, 3.5),
 (4189, 2.5),
 (110, 2.5),
 (2349, 3.0),
 (2285, 3.5),
 (2623, 3.0),
 (1001, 4.0),
 (1490, 3.0),
 (171, 4.0),
 (465, 4.0),
 (733, 5.0),
 (894, 3.0),
 (3771, 3.0),
 (933, 3.0),
 (1083, 3.0),
 (3003, 3.0),
 (11, 3.0),
 (756, 2.5),
 (604, 3.5),
 (258, 4.0),
 (725, 5.0),
 (320, 3.5),
 (1838, 5.0),
 (383, 3.0),
 (3977, 3.0),
 (19

In [89]:
trainset.to_raw_uid(0) # raw_uid means original userid that coming from the orginal dataset

'248'

In [90]:
trainset.to_raw_iid(0) # raw_uid means original title that coming from the orginal dataset

'Life of Pi (2012)'

==================================================================================

##This part of code is only show the mapping (between DF and surprise dataset) for understanding

In [91]:
#  Step 1: Build trainset (already done earlier)
newset = trainset  # Usually this is after trainset = train_test_split(...)[0].build_full_trainset()

#  Step 2: Create inner-to-raw mappings (optional, for reverse lookup or debugging)
# uid_map = {newset.to_inner_uid(uid): uid for uid in newset._raw2inner_id_users}
# iid_map = {newset.to_inner_iid(iid): iid for iid in newset._raw2inner_id_items}

#  Step 3: Extract only the userId-title pairs used in trainset
train_records = [(newset.to_raw_uid(u), newset.to_raw_iid(i)) for (u, i, _) in newset.all_ratings()]
train_df = pd.DataFrame(train_records, columns=['userId', 'title'])

#  Step 4: Join with the original DataFrame to get ratings
df_train_only = pd.merge(train_df, df, on=['userId', 'title'], how='left')

#  Step 5: Add inner UID and IID using newset mappings
df_train_only['uid'] = df_train_only['userId'].apply(lambda x: newset.to_inner_uid(x))
df_train_only['iid'] = df_train_only['title'].apply(lambda x: newset.to_inner_iid(x))

#  Step 6: Reorder for clarity
final_df = df_train_only[['userId', 'uid', 'title', 'iid', 'rating']]

#  Output
print(final_df)


       userId   uid                                title   iid  rating
0         248     0                    Life of Pi (2012)     0     3.0
1         248     0                       WALLÂ·E (2008)   195     4.0
2         248     0         Step Up 2 the Streets (2008)  1066     3.5
3         248     0  Talk to Her (Hable con Ella) (2002)   999     3.5
4         248     0              Illusionist, The (2006)   237     3.0
...       ...   ...                                  ...   ...     ...
365636  84115  2826          Flags of Our Fathers (2006)  1913     3.0
365637  84115  2826                        Snatch (2000)   660     4.0
365638  84115  2826                 Terminal, The (2004)   346     2.5
365639  84115  2826          Boondock Saints, The (2000)   209     4.0
365640  84115  2826       40-Year-Old Virgin, The (2005)   878     3.0

[365641 rows x 5 columns]


In [92]:
final_df

,userId,uid,title,iid,rating
0,248,0,Life of Pi (2012),0,3.0
1,248,0,WALLÂ·E (2008),195,4.0
2,248,0,Step Up 2 the Streets (2008),1066,3.5
3,248,0,Talk to Her (Hable con Ella) (2002),999,3.5
4,248,0,"Illusionist, The (2006)",237,3.0
...,...,...,...,...,...
365636,84115,2826,Flags of Our Fathers (2006),1913,3.0
365637,84115,2826,Snatch (2000),660,4.0
365638,84115,2826,"Terminal, The (2004)",346,2.5
365639,84115,2826,"Boondock Saints, The (2000)",209,4.0


============================================================================================================


In [93]:
# Example: Show how user and item IDs are mapped internally
print("Internal user id 0 maps to:", trainset.to_raw_uid(0))
print("Internal item id 0 maps to:", trainset.to_raw_iid(0))

Internal user id 0 maps to: 248
Internal item id 0 maps to: Life of Pi (2012)


In [94]:
# Import algorithm and metrics
from surprise import KNNWithMeans,accuracy,Prediction

In [95]:
# IBCF - pearson correlation
# UBCf - cosine

# Initialize IBCF algorithm using Pearson correlation (item-based = user_based=False)

algo = KNNWithMeans(k=51, sim_options={'name' : 'pearson' , 'user_based' : True})

In [96]:
# Train the model
algo.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [97]:
len(testset)

121868

In [98]:
# Evaluate model on testset (predict only known interactions)
test_pred = algo.test(testset)

In [99]:
# Calculate RMSE to evaluate prediction accuracy
accuracy.rmse(test_pred)

RMSE: 0.8249


0.8248651737481033

In [100]:
# View some test predictions
test_pred[0:10]

[Prediction(uid='107317', iid='Signs (2002)', r_ui=2.5, est=2.559322042430434, details={'actual_k': 51, 'was_impossible': False}),
 Prediction(uid='103061', iid='Inconvenient Truth, An (2006)', r_ui=4.5, est=4.138221131038445, details={'actual_k': 51, 'was_impossible': False}),
 Prediction(uid='84115', iid='Battlefield Earth (2000)', r_ui=2.5, est=1.1586848601951283, details={'actual_k': 51, 'was_impossible': False}),
 Prediction(uid='130756', iid='Fast and the Furious: Tokyo Drift, The (Fast and the Furious 3, The) (2006)', r_ui=2.0, est=2.4444235613918037, details={'actual_k': 51, 'was_impossible': False}),
 Prediction(uid='24878', iid='Drive (2011)', r_ui=4.5, est=4.472945024567163, details={'actual_k': 51, 'was_impossible': False}),
 Prediction(uid='137648', iid='Matrix Reloaded, The (2003)', r_ui=4.5, est=3.7277530648963824, details={'actual_k': 51, 'was_impossible': False}),
 Prediction(uid='52242', iid='Spy Game (2001)', r_ui=2.5, est=3.9299012496258783, details={'actual_k': 51,

#### Convert predictions to DataFrame for inspection


In [101]:
test_pred_df = pd.DataFrame(test_pred)

In [102]:
test_pred_df.head()

,uid,iid,r_ui,est,details
0,107317,Signs (2002),2.5,2.559322,"{'actual_k': 51, 'was_impossible': False}"
1,103061,"Inconvenient Truth, An (2006)",4.5,4.138221,"{'actual_k': 51, 'was_impossible': False}"
2,84115,Battlefield Earth (2000),2.5,1.158685,"{'actual_k': 51, 'was_impossible': False}"
3,130756,"Fast and the Furious: Tokyo Drift, The (Fast a...",2.0,2.444424,"{'actual_k': 51, 'was_impossible': False}"
4,24878,Drive (2011),4.5,4.472945,"{'actual_k': 51, 'was_impossible': False}"


#### Extract 'was_impossible' flag from prediction details


In [103]:
test_pred_df['was_impossible'] = [ x['was_impossible'] for x in test_pred_df['details']]

In [104]:
test_pred_df.head()

,uid,iid,r_ui,est,details,was_impossible
0,107317,Signs (2002),2.5,2.559322,"{'actual_k': 51, 'was_impossible': False}",False
1,103061,"Inconvenient Truth, An (2006)",4.5,4.138221,"{'actual_k': 51, 'was_impossible': False}",False
2,84115,Battlefield Earth (2000),2.5,1.158685,"{'actual_k': 51, 'was_impossible': False}",False
3,130756,"Fast and the Furious: Tokyo Drift, The (Fast a...",2.0,2.444424,"{'actual_k': 51, 'was_impossible': False}",False
4,24878,Drive (2011),4.5,4.472945,"{'actual_k': 51, 'was_impossible': False}",False


#### Show count of possible vs impossible predictions


In [105]:
test_pred_df['was_impossible'].value_counts()

,count
was_impossible,
False,121417
True,451


#### Make a prediction for a specific user and item


In [106]:
example_pred = algo.predict(uid='41891', iid="Wrong Trousers, The (1993)")
print("Estimated rating:", example_pred.est)

Estimated rating: 3.511396303620614


# ---------------------------------------------
# 💡 Recommend movies not yet rated by a user
# ---------------------------------------------

# Build anti-testset (all user-item pairs NOT in trainset)


####What is build_anti_testset()?
* It builds a testset of all user-item pairs that are NOT in the training set.

* In other words, it creates a list of (user, item, rating) triples for every user-item combination where the user has NOT rated that item in the training data.

* The rating for these pairs is set to the trainset’s global mean rating (just a placeholder).

In [107]:
testset_new = trainset.build_anti_testset()

In [108]:
testset_new[0:5]

[('248', 'Disturbia (2007)', 3.511396303620614),
 ('248', 'Hamlet 2 (2008)', 3.511396303620614),
 ('248', 'Unbreakable (2000)', 3.511396303620614),
 ('248', 'Finding Neverland (2004)', 3.511396303620614),
 ('248', 'X2: X-Men United (2003)', 3.511396303620614)]

In [109]:
len(testset_new)

17308818

#### Predict ratings for unseen movies for first 10,000 user-item pairs


* when we call this algo.test(testset_new[0:10000]) , weighted sum will be used when data points have values and for missing values its the global mean




In [110]:
predictions = algo.test(testset_new[0:10000])

In [111]:
predictions[0]

Prediction(uid='248', iid='Disturbia (2007)', r_ui=3.511396303620614, est=3.3499077359758407, details={'actual_k': 51, 'was_impossible': False})

####What does was_impossible=True mean?
* It means the prediction could NOT be made by the algorithm for that user-item pair.

* This usually happens when the model does not have enough information to produce a meaningful prediction.

####What happens in practice?
* When was_impossible=True, the predicted rating (est) is often set to some default value (like the global mean) or NaN.




In [112]:
# Create DataFrame of predictions
predictions_df = pd.DataFrame([[x.uid, x.iid, x.est] for x in predictions],
                              columns=["userId", "movie_name", "est_rating"])

In [113]:
predictions_df.head()

,userId,movie_name,est_rating
0,248,Disturbia (2007),3.349908
1,248,Hamlet 2 (2008),2.667359
2,248,Unbreakable (2000),3.449773
3,248,Finding Neverland (2004),3.895767
4,248,X2: X-Men United (2003),3.749788


In [114]:
# Sort by user and estimated rating in descending order
predictions_df.sort_values(by=["userId", "est_rating"], ascending=False, inplace=True)


In [115]:
predictions_df.head(30)

,userId,movie_name,est_rating
8580,45844,Shifty (2008),5.000000
8641,45844,BlinkyTM (2011),5.000000
9152,45844,Phish: Bittersweet Motel (2000),5.000000
9413,45844,Dog Pound (2010),5.000000
9778,45844,Frozen Planet (2011),5.000000
9802,45844,"Child I Never Was, The (Leben lang kurze Hosen...",5.000000
9912,45844,Into Eternity (2010),5.000000
9553,45844,"Like Father, Like Son (Soshite chichi ni naru)...",4.993386
9147,45844,Lucky Break (2001),4.971207
9945,45844,"Hollow Crown, The (2012)",4.960816


In [116]:
#  Extract top 10 recommended movies for each user
top_10_recos = predictions_df.groupby('userId').head(10).reset_index(drop=True)


In [117]:
# Show final top 10 recommendations for all users
top_10_recos.head(50)

,userId,movie_name,est_rating
0,45844,Shifty (2008),5.000000
1,45844,BlinkyTM (2011),5.000000
2,45844,Phish: Bittersweet Motel (2000),5.000000
3,45844,Dog Pound (2010),5.000000
4,45844,Frozen Planet (2011),5.000000
5,45844,"Child I Never Was, The (Leben lang kurze Hosen...",5.000000
6,45844,Into Eternity (2010),5.000000
7,45844,"Like Father, Like Son (Soshite chichi ni naru)...",4.993386
8,45844,Lucky Break (2001),4.971207
9,45844,"Hollow Crown, The (2012)",4.960816


In [118]:
# Save recommendations to a CSV file if needed
top_10_recos.to_csv('top_10_recommendations.csv', index=False)